1. [Import](#1)
2. [Data Load](#2)    
3. [Preprocessing](#3)
4. [Model](#4)
5. [Home work](#5)   
    5.1 [prefilter & postfilter](#5.1)    
    5.2 [similar items & similar users](#5.2)   
    5.3 [MainRecommender](#5.3)   
6. [Вывод](#5)   

---

<a id='1'></a>   
__1. Import__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import *

<a id='2'></a>   
__2. Data Load__

In [2]:
data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)




<a id='3'></a>   
__3. Preprocessing__

In [3]:
data_full = data.set_index('item_id').join(item_features.set_index('item_id')).reset_index()

In [4]:
test_size_weeks = 3

data_train = data_full[data_full['week_no'] < data_full['week_no'].max() - test_size_weeks]
data_test = data_full[data_full['week_no'] >= data_full['week_no'].max() - test_size_weeks]

data_train.head(2)

,item_id,user_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,1228,29046618323,157,1,3.49,3313,0.0,2213,23,0.0,0.0,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,25671,358,30707611686,247,1,3.49,3266,0.0,1211,36,0.0,0.0,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 861272, 865456, 86946..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


<a id='4'></a>   
__4. Model__

In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
idmapper = IdMapper(user_item_matrix)

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [9]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=False)


CPU times: user 1min 2s, sys: 35.8 s, total: 1min 38s
Wall time: 16.5 s


-----

__Домашнее задание__

1. Изучите структуру модуля src __+__
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py __+__
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py. __+__  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение. __+__
В prefilter нужно реализовать закомментированные ограничения. __+__
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их в качетсве методов класса. __+__    
Посчитайте с помощью них рекоменадции и найдите precision@5Relf __+__
5. Проверьте, что все модули корректно импортируются __+__
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте __+__

----

<a id='5.1'></a>   
__prefilter_items & postfilter_items__

In [10]:
def prefilter_items(data_train, cut_length=3, sales_range=(10,100)):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
   
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 

    # Уберем самые популярные 
    # Уберем самые непопулряные 
    top_5000_cut = top_5000[cut_length:-cut_length]
    data_train.loc[~data_train['item_id'].isin(top_5000_cut), 'item_id'] = 999999 
    #data_train = data_train.loc[data_train['item_id'] != 999999]

    # Уберем товары, которые не продавались за последние 12 месяцев
    data_train = data_train[data_train['week_no'] > data_train['week_no'].max() - 52]
    
    # Уберем не интересные для рекоммендаций категории (department)
    to_del = ['GROCERY', 'MISC. TRANS.', 'PASTRY', 
              'DRUG GM', 'MEAT-PCKGD',
               'SEAFOOD-PCKGD', 'PRODUCE', 
              'NUTRITION', 'DELI', 'COSMETICS']
    data_train = data_train[~(data_train['department'].isin(to_del))]
    
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    data_train = data_train[~(data_train['sales_value'] < sales_range[0])]
    
    # Уберем слишком дорогие товарыs
    data = data_train[~(data_train['sales_value'] > sales_range[1])]

    return data_train

def postfilter_items(data_train):
    return data_train.loc[data_train['item_id'] != 999999]


<a id='5.2'></a>   
__similar_items_recommendation & similar_users_recommendation__

In [11]:
def get_recommendations(user, model, sparse_user_item, id_mapper, filter_, N=5):
    """Рекомендуем топ-N товаров"""

    res = [id_mapper.id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=user, 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=filter_,  # !!! 
                                    recalculate_user=True)]

    return res





def get_similar_items_recommendation(user_id, model, user_item_matrix, id_mapper,N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров""" 
        user = id_mapper.userid_to_id[user_id]

        top_items = user_item_matrix.toarray()[user, :]
        top_items = np.argsort(-top_items)
        top_items = top_items[:N]
        res = []
        for item in top_items:
            similar_item = [val[0] for val in model.similar_items(itemid=item, N=2) if val[0] != item]
            res.append(id_mapper.id_to_itemid[similar_item[0]])

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res



def get_similar_users_recommendation(user, model, user_item_matrix, id_mapper,  N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    sparse_user_item=csr_matrix(user_item_matrix).T.tocsr()
    res = []
    similar_users = model.similar_users(user, N=N+1)
    filter_ = [999999]
    for u in similar_users:
        if u[0] == user: continue
        recs = get_recommendations(user, model, sparse_user_item, id_mapper, filter_, 1)
        res.append(recs[0])
        filter_.append(id_mapper.itemid_to_id[recs[0]])
    return res





In [12]:
get_similar_items_recommendation(111, model, user_item_matrix, idmapper, N=5)

[1030844, 922289, 5589150, 7133811, 13039201]

In [13]:
get_similar_users_recommendation(111, model, user_item_matrix, idmapper, N=5)

[34873, 73207, 59666, 57390, 244960]

<a id='5.3'></a>   
__MainRecommender__

In [14]:
class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        self.data = data
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = MainRecommender.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров""" 
        user_id = self.userid_to_id[user]

        top_items = self.user_item_matrix.toarray()[user_id, :]
        top_items = np.argsort(-top_items)
        top_items = top_items[:N]

        similar_items = []
        for item in top_items:
            similar_item = [val[0] for val in self.model.similar_items(itemid=item, N=2) if val[0] != item]
            similar_items.append(self.id_to_itemid[similar_item[0]])

        assert len(similar_items) == N, 'Количество рекомендаций != {}'.format(N)
        return similar_items
    
    def get_similar_users_recommendation(self, user_id, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        similar_users = self.model.similar_users(userid=self.userid_to_id[user_id], N=N+1)
        similar_items = []

        for user in similar_users:
            user = user[0]
            if self.userid_to_id[user_id] == user: continue
            top_items = self.user_item_matrix.toarray()[user, :]
            top_items = np.argsort(-top_items)[:N]
            for i in set(np.array(top_items).flatten()):
                if i not in similar_items:
                    similar_items.append(i)
                    break
        assert len(similar_items) == N, 'Количество рекомендаций != {}'.format(N)
        return similar_items

In [15]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
import src.recommenders as MyRecs



In [16]:
mr = MyRecs.MainRecommender(data_train)

In [17]:
user_id = 1
actual = result.loc[result["user_id"] == user_id, "actual"]
similar_items = mr.get_similar_items_recommendation(user_id)
similar_users = mr.get_similar_users_recommendation(user_id)



In [18]:
precision_at_k(similar_items, actual.values[0], k=5)

0.0

In [19]:
precision_at_k(similar_users, actual.values[0], k=5)

0.0

<a id='6'></a>   
__Вывод__   
Не получилось для похожих товаров и похожих пользователей получить precisions > 0. Что видится логичным, т.к. мы рекомендуем не те товарые которые пользователь купил, а похожие.   
Тогда метрика должна быть другая. Получается в этой метрике вообще не должны учавствовать купленные товары.    Или валидировать надо когда пользователь купит или не купит товары. Но тогда и модель надо постоянно переобучать.    
Интересно и непонятно что делать)